# Gallery Example: M/Erl/1 Tandem Network

This example demonstrates a simple 2-queue tandem network:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Erlang at both queues
- **Servers**: 1 server per queue
- **Scheduling**: FCFS

This is a special case of the linear network with n=2.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_merl1_tandem(): Network {
    """Create M/Erl/1-Tandem queueing network (2 queues in series)"""
    val model = Network("M/Erl/1-Tandem")
// Block 1: nodes
    val source = Source(model, "mySource")
    val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
    val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)
    val sink = Sink(model, "mySink")
// Block 2: classes
    val oclass = OpenClass(model, "myClass")
// Exponential arrivals
    source.setArrival(oclass, Exp(1))
// Erlang service with increasing order
    queue1.setService(oclass, Erlang.fit_mean_and_order(0.1, 1))
    queue2.setService(oclass, Erlang.fit_mean_and_order(0.1, 2))
// Block 3: topology
    model.link(Network.serialRouting(source, queue1, queue2, sink))
    
    return model
// Create the model
val model = gallery_merl1_tandem()

## About M/Erl/1 Tandem

This 2-queue tandem model has:
- **Queue 1**: Erlang(1) service = Exponential (SCV = 1.0)
- **Queue 2**: Erlang(2) service (SCV = 0.5)

The decreasing variability at the second queue means:
- Output of Queue 1 is more regular than its input
- Queue 2 experiences less variability
- This can lead to overall better performance than having high variability at both stations

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)